
---

# 🧩 11. **FastAPI with Databases**

FastAPI supports **both relational (SQL)** and **NoSQL (Mongo)** databases using **ORMs and async DB libraries** — crucial for any real-world backend.

---

## 🔹 11.1 SQLModel / SQLAlchemy ORM

* SQLModel (built on SQLAlchemy + Pydantic) is **clean and modern**.
* Great for defining **tables + schemas** in one go.

```python
from sqlmodel import SQLModel, Field

class User(SQLModel, table=True):
    id: int = Field(default=None, primary_key=True)
    name: str
    email: str
```

🔁 Use with SQLAlchemy's session:

```python
from sqlmodel import Session, create_engine

engine = create_engine("sqlite:///db.sqlite")
SQLModel.metadata.create_all(engine)
```

---

## 🔹 11.2 Async DB Layer (`databases` package)

✅ For true **async operations**, use the [`databases`](https://pypi.org/project/databases/) library.

```python
import databases

DATABASE_URL = "sqlite+aiosqlite:///./test.db"
database = databases.Database(DATABASE_URL)

@app.on_event("startup")
async def connect_db():
    await database.connect()

@app.on_event("shutdown")
async def disconnect_db():
    await database.disconnect()
```

---

## 🔹 11.3 MongoDB with `motor` or `beanie`

### `motor` (low-level async MongoDB driver):

```python
from motor.motor_asyncio import AsyncIOMotorClient

client = AsyncIOMotorClient("mongodb://localhost:27017")
db = client["mydb"]
```

### `beanie` (high-level ODM with Pydantic models):

```python
from beanie import Document, init_beanie
from motor.motor_asyncio import AsyncIOMotorClient

class User(Document):
    name: str
    email: str

@app.on_event("startup")
async def init():
    client = AsyncIOMotorClient("mongodb://localhost:27017")
    await init_beanie(database=client.mydb, document_models=[User])
```

---

## 🔹 11.4 CRUD + Dependency Injection

Standard pattern for APIs:

```python
from fastapi import Depends

def get_db():
    with Session(engine) as session:
        yield session

@app.get("/users/{user_id}")
def get_user(user_id: int, db: Session = Depends(get_db)):
    return db.get(User, user_id)
```

---

## ✅ Summary Table

| Tech                 | Use Case                           |
| -------------------- | ---------------------------------- |
| SQLModel             | Modern SQL ORM (sync)              |
| SQLAlchemy           | Industry-standard ORM              |
| `databases`          | Async DB access (Postgres, SQLite) |
| `motor`              | Async MongoDB driver (low-level)   |
| `beanie`             | Pydantic + Mongo ODM (high-level)  |
| Dependency Injection | For clean and testable DB logic    |

---

